In [3]:
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv,CB_target_encoding,LOO_target_encoding,label_encoder
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pd.options.display.float_format = '{:.4f}'.format

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.



In [35]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
training='training'
label='interest_integer'
regression=LogisticRegression(random_state=0,solver='saga',multi_class='auto',n_jobs=-1,fit_intercept=True)
steps = [('scaler', StandardScaler()), ('Log', regression)]
pipeline = Pipeline(steps)
clf=pipeline
df,columns,good_encoders,dp_table,df_tokenizer=load_object(clf.__class__.__name__+'_thesis')
#dp_table

In [36]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

#'l1', 'l2', 'elasticnet', 'none'
def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params = {
       
        'C': trial.suggest_categorical('C', [0.001,0.01,0.1,1,10,100,1000]),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', 'none'])
    }
    print(params)
    regression=LogisticRegression(random_state=0,max_iter=1000,l1_ratio=0.5,solver='saga',multi_class='auto',n_jobs=-1,fit_intercept=True,**params)
    steps = [('scaler', StandardScaler()), ('Log', regression)]
    pipeline = Pipeline(steps)
    clf=pipeline
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))+"log"
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss
search_space = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
   'penalty': ['l1', 'l2', 'elasticnet','none']}


In [ ]:
n=1
for i in search_space.values():
    n*=len(i)
sampler=optuna.samplers.GridSampler(search_space)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:111: ExperimentalWarning:

GridSampler is experimental (supported from v1.2.0). The interface can change in the future.

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'C': 1000, 'penalty': 'l2'}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)

/home/martas/anaconda3/envs/mirko/lib/python3.7/

[I 2020-06-01 20:56:27,685] Finished trial#0 with value: 0.5645375283118628 with parameters: {'C': 1000, 'penalty': 'l2'}. Best is trial#0 with value: 0.5645375283118628.
{'C': 0.01, 'penalty': 'elasticnet'}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



...

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packa

[I 2020-06-01 23:37:48,604] Finished trial#7 with value: 0.564537517518405 with parameters: {'C': 0.001, 'penalty': 'none'}. Best is trial#4 with value: 0.5644978144705782.
{'C': 100, 'penalty': 'elasticnet'}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py

[I 2020-06-02 00:13:57,249] Finished trial#8 with value: 0.5645368002693754 with parameters: {'C': 100, 'penalty': 'elasticnet'}. Best is trial#4 with value: 0.5644978144705782.
{'C': 10, 'penalty': 'none'}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning:

Setting penalty='none' will ignore the C and l1_ratio parameters



In [ ]:
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results.csv')

In [39]:
table

,C,penalty,model_id,score
0,1000.0000,l2,Pipeline_018d86_0.565log,0.5645
1,0.0100,elasticnet,Pipeline_d7b889_0.57log,0.5696
2,0.1000,l1,Pipeline_9af23f_0.565log,0.5655
3,0.1000,l2,Pipeline_a60a3e_0.565log,0.5647
4,1.0000,elasticnet,Pipeline_0b4b26_0.564log,0.5645
5,0.0010,elasticnet,Pipeline_15b62e_0.604log,0.6040
6,0.0010,l1,Pipeline_63823a_0.629log,0.6292
7,0.0010,none,Pipeline_cd5043_0.565log,0.5645
8,100.0000,elasticnet,Pipeline_57bf7d_0.565log,0.5645
9,10.0000,none,Pipeline_12f414_0.565log,0.5645


In [ ]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

#'l1', 'l2', 'elasticnet', 'none'
def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params = {
         'l1_ratio': trial.suggest_uniform('l1_ratio', 0.2, 0.95)
       # 'C': trial.suggest_categorical('C', [0.001,0.01,0.1,1,10,100,1000]),
        #'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', 'none'])
    }
    print(params)
    regression=LogisticRegression(random_state=0,max_iter=1000,C=1,penalty='elasticnet',solver='saga',multi_class='auto',n_jobs=-1,fit_intercept=True,**params)
    steps = [('scaler', StandardScaler()), ('Log', regression)]
    pipeline = Pipeline(steps)
    clf=pipeline
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))+"log_elastic_net"
    trial.set_user_attr('model_id',model_id)
    #save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss

n=5
sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_results_elastic_net.csv')

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



{'l1_ratio': 0.6116101279454935}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py

[I 2020-06-02 15:41:41,337] Finished trial#0 with value: 0.5644949850379032 with parameters: {'l1_ratio': 0.6116101279454935}. Best is trial#0 with value: 0.5644949850379032.
{'l1_ratio': 0.7363920247793145}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py

[I 2020-06-02 16:15:10,822] Finished trial#1 with value: 0.5644941446381778 with parameters: {'l1_ratio': 0.7363920247793145}. Best is trial#1 with value: 0.5644941446381778.
{'l1_ratio': 0.6520725320537328}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py

[I 2020-06-02 16:48:35,420] Finished trial#2 with value: 0.56449449136822 with parameters: {'l1_ratio': 0.6520725320537328}. Best is trial#1 with value: 0.5644941446381778.
{'l1_ratio': 0.6086623872476726}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [5]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

df= pd.read_csv('ensemble_predictions.csv')
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
columns=df.drop(columns=['target','training','listing_id']).columns
training='training'
label='target'

def objective(df,columns,training,label,df_test_listingID,df_train_listingID,trial):
    params = {
         'l1_ratio': trial.suggest_uniform('l1_ratio', 0.2, 0.95),
        'C': trial.suggest_categorical('C', [0.001,0.01,0.1,1]),
        'penalty': trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', 'none'])
    }
    print(params)
    

    clf=LogisticRegression(random_state=0,max_iter=1000,solver='saga',multi_class='auto',n_jobs=-1,fit_intercept=True,**params)
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf )
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id=clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    #save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID,dir='Thesis_predictions_ensemble/')
    return lloss

n=5

sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y

table.to_csv(clf.__class__.__name__+'_grid_LOG_Ensemble.csv')

{'l1_ratio': 0.6116101279454935, 'C': 0.01, 'penalty': 'l1'}


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1501: UserWarning:

l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)



KeyboardInterrupt: 

In [ ]:
#'l1', 'l2', 'elasticnet', 'none'
